In [2]:
import numpy as np
import os
# import pandas as pd
import matplotlib.pyplot as plt
import time, random
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.model_selection import train_test_split
import datetime
import argparse
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader

from ember_utils import *
from ember_model import *
from ember_pjr_utils import *

In [3]:
raw_path = '/home/mr6564/continual_research/AZ_Data/Domain/'
train_file = raw_path + 'All_Domain_AZ_Train.npz'
test_file = raw_path + 'All_Domain_AZ_Test.npz'


train_data = np.load(train_file, allow_pickle=True)
test_data = np.load(test_file, allow_pickle=True)

X_train, Y_train, Y_tr_family_year = train_data['X_train'], train_data['Y_train'], train_data['Y_tr_family']
X_test, Y_test, Y_te_family_year = test_data['X_test'], test_data['Y_test'], test_data['Y_te_family']


In [12]:
class Ember_MLP_Net(nn.Module):
    def __init__(self, input_features):
        super(Ember_MLP_Net, self).__init__()
        
        self.fc1 = nn.Linear(input_features, 1024)
        #self.fc1_bn = nn.BatchNorm1d(1024)
        self.act1 = nn.ReLU()
        #self.fc1_drop = nn.Dropout(p=0.5)
        
        self.fc2 = nn.Linear(1024, 512)
        #self.fc2_bn = nn.BatchNorm1d(512)
        self.act2 = nn.ReLU()
        #self.fc2_drop = nn.Dropout(p=0.5)
        
        self.fc3 = nn.Linear(512, 256)
        #self.fc3_bn = nn.BatchNorm1d(256)
        self.act3 = nn.ReLU()
        #self.fc3_drop = nn.Dropout(p=0.5)        
        
        self.fc4 = nn.Linear(256, 128)
        #self.fc4_bn = nn.BatchNorm1d(128)
        self.act4 = nn.ReLU()
        #self.fc4_drop = nn.Dropout(p=0.5)  
        
        self.fc_last = nn.Linear(128, 1) 
        self.out = nn.Sigmoid()
        
        #self.activate = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        #print(x.shape)
        x = self.fc1(x)
        #x = self.fc1_bn(x)
        x = self.act1(x) 
        #x = self.fc1_drop(x)

        x = self.fc2(x)
        #x = self.fc2_bn(x)
        x = self.act2(x) 
        #x = self.fc2_drop(x)
        
        x = self.fc3(x)
        #x = self.fc3_bn(x)
        x = self.act3(x) 
        #x = self.fc3_drop(x)
        
        x = self.fc4(x)
        #x = self.fc4_bn(x)
        x = self.act4(x)
        #x = self.fc4_drop(x)
        
        x = self.fc_last(x)
        x = self.out(x)
        return x






exp_seeds = [random.randint(1, 99999) for i in range(1)]


accs_all = []
rocauc_all = []

num_epoch = 500
batch_size = 512
patience = 5


input_features = X_train.shape[1]

replay_type, current_task = 'azdomain', 'azdomain'

for exp in exp_seeds:

    start_time = time.time()
    use_cuda = True
    print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
    use_cuda = use_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    torch.manual_seed(exp)

    model = Ember_MLP_Net(input_features)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.000001)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
       
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    
    model = model.to(device)
    print(f'Model has {count_parameters(model)/1000000}m parameters')    
    criterion = nn.BCELoss()    

    
#     standardization = StandardScaler()
#     standard_scaler = standardization.fit(X_train)

#     X_train = standard_scaler.transform(X_train)
#     X_test = standard_scaler.transform(X_test)
    
    X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
    X_test, Y_test = np.array(X_test, np.float32), np.array(Y_test, np.int32)  

    
    model_save_dir = '../az_model/model/'
    create_parent_folder(model_save_dir)

    opt_save_path = '../az_model/opt/'
    create_parent_folder(opt_save_path)

    results_save_dir =  '../az_model/res/' 
    create_parent_folder(results_save_dir)

    print(f'X_train {X_train.shape} Y_train {Y_train.shape}')
    print(f'X_test {X_test.shape} Y_test {Y_test.shape}')
    
    
    task_training_time, epoch_ran, training_loss, validation_loss  = training_early_stopping(\
                                 model, model_save_dir, opt_save_path, X_train, Y_train,\
                                 X_test, Y_test, patience, batch_size, device, optimizer, num_epoch,\
                                 criterion, replay_type, current_task, exp, earlystopping=True)

    
    end_time = time.time()
    print(f'Elapsed time {(end_time - start_time)/60} mins.') 
    
    
    
    best_model_path = model_save_dir + os.listdir(model_save_dir)[0]
    print(f'loading best model {best_model_path}')
    model.load_state_dict(torch.load(best_model_path))

    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.000001)
    best_optimizer = opt_save_path + os.listdir(opt_save_path)[0]
    print(f'loading best optimizer {best_optimizer}')
    optimizer.load_state_dict(torch.load(best_optimizer))


    acc, precision, recall, f1score = testing_aucscore(model, X_test, Y_test, batch_size, device)
    print()
    del model_save_dir
    del opt_save_path
    del results_save_dir
    
#     accs_all.append(acc)
#     rocauc_all.append(rocauc)



Torch 2.0.1 CUDA 11.8
Model has 3.720193m parameters
X_train (682598, 2959) Y_train (682598,)
X_test (75848, 2959) Y_test (75848,)
Epoch 1 of 500


100%|████████████████████████████████████████████████████████████| 149/149 [00:01<00:00, 112.49it/s]


Train Loss: 0.6893, Train Acc: 0.5271
Val Loss: 0.6989, Val Acc: 0.5849
Validation loss decreased (inf --> 0.698858).  Saving model ...
../az_model/model/best_model_epoch_1.pt
../az_model/opt/best_optimizer_epoch_1.pt
Epoch 2 of 500


100%|█████████████████████████████████████████████████████████████| 149/149 [00:01<00:00, 92.53it/s]


Train Loss: 0.6668, Train Acc: 0.5739
Val Loss: 0.6650, Val Acc: 0.5801
Validation loss decreased (0.698858 --> 0.664996).  Saving model ...
../az_model/model/best_model_epoch_2.pt
../az_model/opt/best_optimizer_epoch_2.pt
Epoch 3 of 500


...

Train Loss: 0.4871, Train Acc: 0.7216
Val Loss: 0.7850, Val Acc: 0.6066
EarlyStopping counter: 4 out of 5
Epoch 7 of 500


100%|████████████████████████████████████████████████████████████| 149/149 [00:01<00:00, 119.26it/s]


Train Loss: 0.4664, Train Acc: 0.7342
Val Loss: 0.7730, Val Acc: 0.6409
EarlyStopping counter: 5 out of 5
Early stopping
Training time: 1.800 minutes
Elapsed time 1.8717517852783203 mins.
loading best model ../az_model/model/best_model_epoch_2.pt
loading best optimizer ../az_model/opt/best_optimizer_epoch_2.pt


100%|█████████████████████████████████████████████████████████████| 149/149 [00:01<00:00, 88.18it/s]

test accuracy 0.5050845334193851 and ROC-AUC 0.5054501448809074

